# GCP Demo 3: Text translation using AutoML

### Import Modules

In [5]:
import os
import shutil

from gcpdemo3 import etl
from gcpdemo3 import predict

from google.oauth2 import service_account

### Set translation and prediction credentials path

In [8]:
# set credentials paths
translation_credentials = '..\credentials\translation\ml-sandbox-1-191918-b473cb40490b.json'
prediction_credentials =  '..\credentials\prediction\ml-sandbox-1-191918-58d6a5f3d5c2.json'

### Run ETL

In [ ]:
# copy raw book files from GCS
os.system('gcloud config set project ml-sandbox-1-191918')
os.mkdir('./temp')
os.system('gsutil -m cp -r gs://gcp-cert-demo-3/opus ./temp')

# process native english books and concatenate
etl.process_book(
    in_path='./temp/opus/Books/raw/en/Austen_Jane-Pride_and_Prejudice.xml',
    out_path='./temp/Austen_Jane-Pride_and_Prejudice_en_processed.xml'
)
etl.process_book(
    in_path='./temp/opus/Books/raw/en/Twain_Mark-Tom_Sawyer.xml',
    out_path='./temp/Twain_Mark-Tom_Sawyer_en_processed.xml'
)
etl.process_book(
    in_path='./temp/opus/Books/raw/en/Doyle_Arthur_Conan-Adventures_of_Sherlock_Holmes.xml',
    out_path='./temp/Doyle_Arthur_Conan-Adventures_of_Sherlock_Holmes_en_processed.xml'
)
etl.concat_label_files(
    in_paths=[
        './temp/Austen_Jane-Pride_and_Prejudice_en_processed.xml',
        './temp/Twain_Mark-Tom_Sawyer_en_processed.xml',
        './temp/Doyle_Arthur_Conan-Adventures_of_Sherlock_Holmes_en_processed.xml'
    ],
    out_path='./temp/native.csv',
    label='native'
)

# process professionally translated books and concatenate
etl.process_book(
    in_path='./temp/opus/Books/raw/en/Cervantes_Miguel-Don_Quijote.xml',
    out_path='./temp/Cervantes_Miguel-Don_Quijote_en_processed.xml'
)
etl.process_book(
    in_path='./temp/opus/Books/raw/en/Hugo_Victor-Notre_Dame_de_Paris.xml',
    out_path='./temp/Hugo_Victor-Notre_Dame_de_Paris_en_processed.xml'
)
etl.process_book(
    in_path='./temp/opus/Books/raw/en/Flaubert_Gustave-Madame_Bovary.xml',
    out_path='./temp/Flaubert_Gustave-Madame_Bovary_en_processed.xml'
)
etl.concat_label_files(
    in_paths=[
        './temp/Cervantes_Miguel-Don_Quijote_en_processed.xml',
        './temp/Hugo_Victor-Notre_Dame_de_Paris_en_processed.xml',
        './temp/Flaubert_Gustave-Madame_Bovary_en_processed.xml'
    ],
    out_path='./temp/translated.csv',
    label='translated'
)

# get translation credentials
trans_credentials = service_account.Credentials.from_service_account_file(translation_credentials)

# process and translate native spanish book
etl.process_book(
    in_path='./temp/opus/Books/raw/es/Cervantes_Miguel-Don_Quijote.xml',
    out_path='./temp/Cervantes_Miguel-Don_Quijote_es_processed.xml'
)
etl.translate_book(
    credentials=trans_credentials,
    in_path='./temp/Cervantes_Miguel-Don_Quijote_es_processed.xml',
    out_path='./temp/cervantes_translated.txt',
    source='es',
    target='en',
    chunk_size=10
)

# process and translate native french book
etl.process_book(
    in_path='./temp/opus/Books/raw/fr/Hugo_Victor-Notre_Dame_de_Paris.xml',
    out_path='./temp/Hugo_Victor-Notre_Dame_de_Paris_fr_processed.xml'
)
etl.translate_book(
    credentials=trans_credentials,
    in_path='./temp/Hugo_Victor-Notre_Dame_de_Paris_fr_processed.xml',
    out_path='./temp/victorhugo_translated.txt',
    source='fr',
    target='en',
    chunk_size=10
)

# concatenate translated books
etl.concat_label_files(
    in_paths=[
        './temp/cervantes_translated.txt',
        './temp/victorhugo_translated.txt'
    ],
    out_path='./temp/machine_translated.csv',
    label='machine'
)

### Train Model

In [ ]:
# TODO: train model

### Remove Temporary Directory

In [ ]:
# remove temporary directory
shutil.rmtree('./temp')

### Deploy Model

In [ ]:
# TODO: deploy model

### Make Predictions

In [11]:
# get prediction credentials
pred_credentials = service_account.Credentials.from_service_account_file(prediction_credentials)

# make predictions using deployed model
predictor = predict.Predictor(
    credentials=pred_credentials,
    model_name='projects/261855689705/locations/us-central1/models/TCN7361261134285897728'
)
predictions = predictor.predict([
    'It is a truth universally acknowledged that a single man in possession of a good fortune must be in want of a wife.',
    'My dear Mr. Bennet said his lady to him one day have you heard that Netherfield Park is let at last?',
    'But it is returned she; for Mrs. Long has just been here and she told me all about it.',
    'Do you not want to know who has taken it? cried his wife impatiently.'
])
predictions

,native_score,transl_score,machine_score,line
0,70.001101,10.983597,19.015306,It is a truth universally acknowledged that a ...
1,99.986970,0.009422,0.003608,My dear Mr. Bennet said his lady to him one da...
2,98.484021,1.294325,0.221653,But it is returned she; for Mrs. Long has just...
3,90.697706,1.600761,7.701537,Do you not want to know who has taken it? crie...
